In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [25]:
df = pd.read_csv('train.csv')
df.shape

(404290, 6)

In [26]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [27]:
new_df = df.sample(30000)

In [28]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [29]:
new_df.duplicated().sum()

0

In [30]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
364376,What questions you'll ask as an interviewer wh...,What question you'll ask as an interviewer whe...
99445,How did you specialize as a software developer?,"I'm interested in digital security, but don't ..."
201750,Why does nobody have time?,Why does nobody like me?
144976,How can I apply Machine Learning to predicting...,What are 10-15 applications of image captionin...
213044,What is the cure for chronic eczema?,How can you cure eczema?


In [31]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [32]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [33]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
364376,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
99445,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201750,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213044,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
234508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
339201,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
387079,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [35]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
364376,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
99445,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201750,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

**Check Accuracy using Random Forest Classifier**

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7448333333333333

**Check Accuracy using XGB Classifier**

In [16]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7235